# Text Analytics - Assignment 5

Grammatikopoulou Maria - f3352310

Phevos A. Margonis - f3352317

Moniaki Melina - f3352321

In [ ]:
#NLP Parsing and Processing - Uncomment if using these libraries<br>
# !pip install transformers<br>
# !pip install datasets<br>
# !pip install accelerate<br>
# !pip install conllu<br>
# !pip install halo<br>
# !pip install pydot<br>
# !pip install fasttext-wheel<br>
# !pip install tensorflow[and-cuda] # For linux & GPU support<br>
import gensim.downloader as api
import pickle
import datetime
import os
import re
import time
import warnings
from collections import Counter
import nltk
# import fasttext
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
# from catalogue import create
from conllu import parse_incr
# from halo import Halo
from keras.utils import to_categorical
from numpy.random import default_rng
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, auc, classification_report,
                             f1_score, precision_recall_curve, precision_score,
                             recall_score)
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from tensorflow.keras.callbacks import (EarlyStopping, ModelCheckpoint,
                                        TensorBoard)
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam
import transformers
transformers.__version__
from tqdm import tqdm

In [41]:
rng = default_rng(587)
#%% Reign-in VRAM use from Tensorflow

% load and explore

In [42]:
trainDF = pd.read_csv('en_gum_train.csv').drop_duplicates().reset_index(drop=True)
valDF = pd.read_csv('en_gum_val.csv')
testDF = pd.read_csv('en_gum_test.csv')

In [43]:
trainDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8165 entries, 0 to 8164
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    8165 non-null   object
 1   labels  8165 non-null   object
dtypes: object(2)
memory usage: 127.7+ KB


In [44]:
trainDF.describe()

,text,labels
count,8165,8165
unique,8160,7331
top,Sure .,INTJ PUNCT
freq,3,61


In [45]:
trainDF.head()
print(pd.Series([len(text.split()) for text in trainDF.text]).describe())
percentile = 90
print(f"{percentile}th Percentile length of train sentences: {np.percentile([len(text.split()) for text in trainDF.text],percentile)}")

count    8165.000000
mean       18.003552
std        13.221574
min         1.000000
25%         8.000000
50%        15.000000
75%        25.000000
max       110.000000
dtype: float64
90th Percentile length of train sentences: 35.0


In [46]:
print(trainDF.text[0])
print(trainDF.labels[0])

Aesthetic Appreciation and Spanish Art :
ADJ NOUN CCONJ ADJ NOUN PUNCT


In [47]:
TARGET_NAMES = sorted(list(set(' '.join(trainDF.labels).split())))
NCLASSES = len(TARGET_NAMES)
print('Target Names:',TARGET_NAMES)

Target Names: ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']


% Majority

In [48]:
def flatten_and_encode(df) -> pd.DataFrame:
    '''Flatten each text sentence and each label. Then Encode the labels as category ints'''
    df_flattened        = df['text'].str.lower().str.split(expand=True).stack().reset_index(drop=True).to_frame(name='word') # Flatten train data
    df_flattened['tag'] = df['labels'].str.lower().str.split(expand=True).stack().reset_index(drop=True)
    df_flattened['tag'] = df_flattened['tag'].astype('category').cat.codes # Encode 'tag'
    return df_flattened

Majority Dummy classifier: Find the most common label for each unique word using the Train-set

In [49]:
train_flattened  = flatten_and_encode(trainDF)
trainMajorityTag = train_flattened.groupby('word')['tag'].agg(lambda x: x.mode()[0]).to_frame() # Find the most common tag for each word
trainMajorityTag = trainMajorityTag.tag.to_dict() # dict{word: most_commont_tag}

Flatten evaluation datasets

In [50]:
val_flattened  = flatten_and_encode(valDF)
test_flattened = flatten_and_encode(testDF)

Get y_true for each dataset

In [51]:
y_train_true = train_flattened.tag.tolist()
y_val_true   = val_flattened.tag.tolist()
y_test_true  = test_flattened.tag.tolist()

Find the overall most common tag inside train-set

In [52]:
most_common_tag = train_flattened.tag.mode()[0]

Create the prediction using Majority dummy classifier

In [53]:
y_train_pred = [trainMajorityTag.get(word,most_common_tag) for word in train_flattened.word]
y_val_pred   = [trainMajorityTag.get(word,most_common_tag) for word in val_flattened.word]
y_test_pred  = [trainMajorityTag.get(word,most_common_tag) for word in test_flattened.word]

In [54]:
majority_val_acc = accuracy_score(y_val_true, y_val_pred).round(2)

In [55]:
print(classification_report(y_train_true, y_train_pred, target_names=TARGET_NAMES))
print(classification_report(y_val_true, y_val_pred, target_names=TARGET_NAMES))
print(classification_report(y_test_true, y_test_pred, target_names=TARGET_NAMES))

              precision    recall  f1-score   support

         ADJ       0.91      0.93      0.92      9761
         ADP       0.90      0.89      0.90     14045
         ADV       0.89      0.85      0.87      6969
         AUX       0.87      0.95      0.91      7689
       CCONJ       0.99      1.00      0.99      4846
         DET       0.95      0.98      0.96     12066
        INTJ       0.73      0.76      0.74      1144
        NOUN       0.92      0.95      0.93     24618
         NUM       0.95      1.00      0.98      2924
        PART       0.69      0.90      0.78      3466
        PRON       0.92      0.96      0.94     12059
       PROPN       0.94      0.82      0.88      8934
       PUNCT       0.99      1.00      1.00     20274
       SCONJ       0.85      0.33      0.48      2324
         SYM       0.95      0.68      0.79       231
        VERB       0.93      0.89      0.91     15347
           X       0.97      0.73      0.84       302

    accuracy              

# Note:
There is an issue with Tensorflow which takes up too much VRAM, so the following workaround must be made.

# Vectorize y<br>
vectorize_Y = layers.TextVectorization(max_tokens=MAX_WORDS,<br>
                                output_mode='int',<br>
                                standardize=None,<br>
                                ngrams=1,<br>
                                vocabulary=TARGET_NAMES,<br>
                                output_sequence_length=MAX_SEQUENCE_LENGTH)

y_train_sequential = vectorize_Y(trainDF.labels.values).numpy()<br>
y_val_sequential = vectorize_Y(valDF.labels.values).numpy()<br>
y_test_sequential = vectorize_Y(testDF.labels.values).numpy()

# Save the arrays<br>
np.save('y_train_sequential.npy', y_train_sequential)<br>
np.save('y_val_sequential.npy', y_val_sequential)<br>
np.save('y_test_sequential.npy', y_test_sequential)

Load the arrays back

In [56]:
y_train_sequential = np.load('y_train_sequential.npy')
y_val_sequential = np.load('y_val_sequential.npy')
y_test_sequential = np.load('y_test_sequential.npy')

% Reformat input data

In [57]:
from datasets.dataset_dict import DatasetDict
from datasets import Dataset # Tropos na diaxeirizese ta data gia na ta dineis stous transformers

Create a dictionary containing X, y, for every subset

In [58]:
d = {'train':Dataset.from_dict({'label':y_train_sequential,'text':trainDF.text.str.split()}),
     'val':Dataset.from_dict({'label':y_val_sequential,'text':valDF.text.str.split()}),
     'test':Dataset.from_dict({'label':y_test_sequential,'text':testDF.text.str.split()})
     }

In [59]:
mydataset = DatasetDict(d)

% Example train instance

In [60]:
print(mydataset['train']['text'][0])
print(mydataset['train']['label'][0])

['Aesthetic', 'Appreciation', 'and', 'Spanish', 'Art', ':']
[2, 9, 6, 2, 9, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


% model & tokenizer

In [61]:
model_name = "distilbert/distilbert-base-uncased"

In [62]:
from transformers import AutoTokenizer

In [63]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

% Tokenization Example

In [64]:
tokenized_input = tokenizer(mydataset["train"][2]["text"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokenized_input, '\n')
print(tokenized_input.word_ids(), '\n')
print(tokens)

{'input_ids': [101, 6249, 8925, 1011, 5811, 6249, 1012, 8925, 1011, 5811, 1030, 3417, 1012, 9353, 1012, 2866, 2118, 1997, 10913, 1010, 2142, 2983, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]} 

[None, 0, 1, 2, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 6, 7, 8, 9, 10, None] 

['[CLS]', 'claire', 'bailey', '-', 'ross', 'claire', '.', 'bailey', '-', 'ross', '@', 'port', '.', 'ac', '.', 'uk', 'university', 'of', 'portsmouth', ',', 'united', 'kingdom', '[SEP]']


In [65]:
tokenized_input = tokenizer(mydataset["train"][113]["text"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokenized_input, '\n')
print(tokenized_input.word_ids(), '\n')
print(tokens)

{'input_ids': [101, 4372, 3900, 14905, 3672, 3138, 2173, 2043, 1037, 19962, 2696, 13306, 3131, 2003, 3714, 2039, 2408, 2048, 3210, 1997, 4623, 1006, 14383, 2075, 29488, 6178, 2906, 7352, 1010, 2456, 1024, 9800, 1007, 1010, 3228, 4125, 2000, 2367, 24828, 3896, 1006, 1041, 1012, 1043, 1012, 3445, 7902, 2006, 3787, 1997, 1996, 3714, 1011, 2039, 7655, 1010, 2030, 5688, 2090, 2216, 3787, 1007, 1010, 2030, 4526, 3313, 15931, 2005, 1996, 4372, 3900, 18552, 2094, 3210, 1006, 7439, 1011, 6643, 6460, 1010, 2889, 1007, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]} 

[None, 0, 0, 0, 0, 1, 2, 3, 4, 5, 5, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 16, 16, 17, 17, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 31, 31, 31, 32, 33, 34, 35, 36, 37, 38, 39, 

% Re-align tokens after BPE

In [76]:
def tokenize_and_align_labels(examples):
    global tokenizer
    ''' 
    Adding the special tokens [CLS] and [SEP] and subword tokenization creates a mismatch between
    the input and labels. A single word corresponding to a single label may be split into two subwords. 
    You will need to realign the tokens and labels by:
        
        1. Mapping all tokens to their corresponding word with the word_ids method.
        2. Assigning the label -100 to the special tokens [CLS] and [SEP] so the loss function ignores them.
        3. Only labeling the first token of a given word. Assign -100 to other subtokens from the same word.
        
    Here is how you can create a function to realign the tokens and labels, and truncate sequences to be no longer than model's maximum input length:
    '''
    tokenized_inputs = tokenizer(examples["text"], truncation=True, 
                                 is_split_into_words=True, 
                                 add_special_tokens=True, 
                                 padding='max_length',
                                 max_length=32, 
                                 return_tensors='pt'
                                 )
    labels = []
    for i, label in enumerate(examples["label"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

Call

In [77]:
myTokenizedDataset = mydataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/8165 [00:00<?, ? examples/s]

Map:   0%|          | 0/1117 [00:00<?, ? examples/s]

Map:   0%|          | 0/1096 [00:00<?, ? examples/s]

In [78]:
myTokenizedDataset['train'][2]

{'label': [13,
  13,
  14,
  13,
  13,
  13,
  3,
  13,
  14,
  17,
  13,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'text': ['Claire',
  'Bailey',
  '-',
  'Ross',
  'claire.bailey-ross@port.ac.uk',
  'University',
  'of',
  'Portsmouth',
  ',',
  'United',
  'Kingdom'],
 'input_ids': [101,
  6249,
  8925,
  1011,
  5811,
  6249,
  1012,
  8925,
  1011,
  5811,
  1030,
  3417,
  1012,
  9353,
  1012,
  2866,
  2118,
  1997,
  10913,
  1010,
  2142,
  2983,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'labels': [-100,
  13,
  13,
  14,
  13,
  13,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  13,
  3,
  13,
  14,
  17,
  13,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100]

In [79]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

In [80]:
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=19).to("cuda")

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [81]:
from sklearn.metrics import f1_score, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids.flatten()  # Flatten the label_ids
    preds = pred.predictions.argmax(-1).flatten()  # Flatten and get the max prediction
    
    mask = (labels != 0)
    
    # Apply mask
    masked_labels = labels[mask]
    masked_preds = preds[mask]

    # Calculate metrics using sklearn's functions
    acc = accuracy_score(masked_labels, masked_preds)
    # f1 = f1_score(masked_labels, masked_preds, average='macro')

    return {
        'val_accuracy': acc,
        # 'val_f1': f1,
    }

% fit

In [82]:
from transformers import EarlyStoppingCallback

In [83]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=20,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    logging_dir='./logs',
    warmup_steps=500,  # Adjust based on your setup
    evaluation_strategy="epoch",  # Evaluation at the end of each epoch
    save_strategy="epoch",  # Align saving strategy with evaluation strategy
    load_best_model_at_end=True,
    save_total_limit=1,
    gradient_accumulation_steps=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=myTokenizedDataset["train"],
    eval_dataset=myTokenizedDataset["val"],
    tokenizer=tokenizer,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,                # the callback that computes metrics of interest
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.01)]  # Add your early stopping callback here
)

trainer.train()

Epoch,Training Loss,Validation Loss,Val Accuracy
0,No log,0.984467,0.424117
2,1.382900,0.486161,0.673382
2,1.382900,0.317175,0.789215
4,0.423100,0.255461,0.831485
4,0.423100,0.223713,0.856722
6,0.268400,0.204747,0.873076
6,0.268400,0.188690,0.881168
8,0.206600,0.178343,0.892429
8,0.206600,0.180714,0.892202
10,0.168300,0.170694,0.899049


TrainOutput(global_step=2810, training_loss=0.45298437614033654, metrics={'train_runtime': 480.9244, 'train_samples_per_second': 339.554, 'train_steps_per_second': 10.605, 'total_flos': 733638448297920.0, 'train_loss': 0.45298437614033654, 'epoch': 11.0})

In [84]:
import torch
from torch.nn.functional import softmax

In [85]:
# Train Classification Report
mask_train = (y_train_sequential.flatten() != 0)
pred_train_probas = softmax(torch.tensor(trainer.predict(myTokenizedDataset["train"]).predictions), dim=-1)
pred_train_labels = np.argmax(pred_train_probas, axis=-1)
print(classification_report(y_train_sequential.flatten()[mask_train], 
                            pred_train_labels.flatten()[mask_train],
                            labels=list(range(2,19)),
                            target_names=TARGET_NAMES
                            ))
# Validation Classification Report
mask_val = (y_val_sequential.flatten() != 0)
pred_val_probas = softmax(torch.tensor(trainer.predict(myTokenizedDataset["val"]).predictions), dim=-1)
pred_val_labels = np.argmax(pred_val_probas, axis=-1)
print(classification_report(y_val_sequential.flatten()[mask_val], 
                            pred_val_labels.flatten()[mask_val],
                            labels=list(range(2,19)),
                            target_names=TARGET_NAMES
                            ))

# Test Classification Report
mask_test = (y_test_sequential.flatten() != 0)
pred_test_probas = softmax(torch.tensor(trainer.predict(myTokenizedDataset["test"]).predictions), dim=-1)
pred_test_labels = np.argmax(pred_test_probas, axis=-1)
print(classification_report(y_test_sequential.flatten()[mask_test], 
                            pred_test_labels.flatten()[mask_test],
                            labels=list(range(2,19)),
                            target_names=TARGET_NAMES
                            ))

              precision    recall  f1-score   support

         ADJ       0.97      0.91      0.94      8907
         ADP       0.95      0.93      0.94     12922
         ADV       0.96      0.92      0.94      6508
         AUX       0.98      0.96      0.97      7266
       CCONJ       0.97      0.92      0.94      4356
         DET       0.98      0.96      0.97     11187
        INTJ       0.96      0.88      0.92      1118
        NOUN       0.90      0.97      0.93     22484
         NUM       0.90      0.88      0.89      2567
        PART       0.98      0.94      0.96      3253
        PRON       0.97      0.96      0.97     11447
       PROPN       0.91      0.92      0.91      8116
       PUNCT       0.94      0.94      0.94     17766
       SCONJ       0.94      0.90      0.92      2183
         SYM       0.90      0.38      0.53       211
        VERB       0.95      0.96      0.95     14428
           X       0.89      0.51      0.65       265

   micro avg       0.94   

              precision    recall  f1-score   support

         ADJ       0.92      0.86      0.89      1187
         ADP       0.92      0.90      0.91      1711
         ADV       0.93      0.86      0.90       848
         AUX       0.96      0.94      0.95       991
       CCONJ       0.95      0.89      0.92       558
         DET       0.95      0.93      0.94      1450
        INTJ       0.90      0.81      0.85       201
        NOUN       0.86      0.93      0.89      3026
         NUM       0.85      0.80      0.82       275
        PART       0.96      0.92      0.94       468
        PRON       0.95      0.95      0.95      1687
       PROPN       0.81      0.78      0.80       746
       PUNCT       0.92      0.92      0.92      2229
       SCONJ       0.87      0.81      0.84       305
         SYM       0.00      0.00      0.00         7
        VERB       0.91      0.92      0.91      1972
           X       1.00      0.09      0.17        11

   micro avg       0.91   

              precision    recall  f1-score   support

         ADJ       0.90      0.83      0.87      1233
         ADP       0.90      0.88      0.89      1917
         ADV       0.89      0.85      0.87       861
         AUX       0.95      0.95      0.95       900
       CCONJ       0.92      0.87      0.90       627
         DET       0.94      0.93      0.93      1633
        INTJ       0.90      0.79      0.84       144
        NOUN       0.84      0.90      0.87      3243
         NUM       0.83      0.84      0.83       365
        PART       0.93      0.92      0.93       393
        PRON       0.95      0.94      0.94      1347
       PROPN       0.79      0.75      0.77      1294
       PUNCT       0.89      0.88      0.89      2236
       SCONJ       0.86      0.85      0.86       241
         SYM       0.89      0.25      0.39        32
        VERB       0.93      0.90      0.91      1962
           X       0.73      0.46      0.56        24

   micro avg       0.89   

In [86]:
# Predict class probabilities to be used for PR-AUC
def my_padded_multiclass_auc_report( y_true, y_pred, title: str, TARGET_NAMES=TARGET_NAMES) -> None:
    """
    Calculate the Precision-Recall AUC for each class, and then their Macro-averaged score.
    The classes are padded with '', and [UNK] classes because they have beed created with 
    TextVectorizer() layer.

    Parameters
    ----------
    y_true : np.array of int
        The one-hot label vectors for each data-point.
    y_pred : np.array of float
        The MxN predicted probabilities for each data-point (M) and class (N).
    title : str
        The title of the data-set that is processed. E.g.: Train/Val/Test.

    Returns
    -------
    None

    """
    print(f"=== PR-AUC for {title} ===")
    macro_auc = []
    for i in range(2,len(TARGET_NAMES)):
        precision, recall, _ = precision_recall_curve(y_true[:, i], y_pred[:, i])
        area = auc(recall, precision)
        print(f"PR-AUC for class {TARGET_NAMES[i]}: {area*100:.2f}%")
        macro_auc.append(area)

    print(f"-> Macro-averaged PR-AUC: {np.mean(macro_auc)*100:.2f}%\n")

In [87]:
# Suppress specific warnings temporarily
warnings.filterwarnings("ignore", message="No positive class found in y_true")

# Predict probas.
# Reshape 3D to 2D (samples, max_sequence, classes) -> (tokens,classes). Then apply mask.
predictions_train = pred_train_probas
predictions_train = predictions_train.reshape(-1, predictions_train.shape[-1])[mask_train]
predictions_val   = pred_val_probas
predictions_val   = predictions_val.reshape(-1, predictions_val.shape[-1])[mask_val]
predictions_test  = pred_test_probas
predictions_test  = predictions_test.reshape(-1, predictions_test.shape[-1])[mask_test]

# PR-AUC report
my_padded_multiclass_auc_report(to_categorical(y_train_sequential.flatten()[mask_train]), predictions_train, 'Train')
my_padded_multiclass_auc_report(to_categorical(y_val_sequential.flatten()[mask_val]), predictions_val, 'Validation')
my_padded_multiclass_auc_report(to_categorical(y_test_sequential.flatten()[mask_test]), predictions_test, 'Test')

=== PR-AUC for Train ===
PR-AUC for class ADV: 98.00%
PR-AUC for class AUX: 98.65%
PR-AUC for class CCONJ: 97.69%
PR-AUC for class DET: 99.07%
PR-AUC for class INTJ: 97.63%
PR-AUC for class NOUN: 99.18%
PR-AUC for class NUM: 97.27%
PR-AUC for class PART: 99.15%
PR-AUC for class PRON: 95.36%
PR-AUC for class PROPN: 98.01%
PR-AUC for class PUNCT: 99.43%
PR-AUC for class SCONJ: 97.58%
PR-AUC for class SYM: 99.11%
PR-AUC for class VERB: 95.64%
PR-AUC for class X: 65.69%
-> Macro-averaged PR-AUC: 95.83%

=== PR-AUC for Validation ===
PR-AUC for class ADV: 94.20%
PR-AUC for class AUX: 96.92%
PR-AUC for class CCONJ: 94.34%
PR-AUC for class DET: 98.35%
PR-AUC for class INTJ: 95.62%
PR-AUC for class NOUN: 98.23%
PR-AUC for class NUM: 93.01%
PR-AUC for class PART: 96.78%
PR-AUC for class PRON: 91.02%
PR-AUC for class PROPN: 97.01%
PR-AUC for class PUNCT: 98.82%
PR-AUC for class SCONJ: 87.58%
PR-AUC for class SYM: 98.24%
PR-AUC for class VERB: 88.67%
PR-AUC for class X: 3.02%
-> Macro-averaged PR